Rewrite training data from original LLM study (Organisciak et al. 2023) to OpenAI chat format, for an exact comparison.

In [2]:
from pathlib import Path
import json

In [10]:
trainpath = Path('../data/ocsai1/finetune-gt_main2_prepared_train.jsonl')
valpath = Path('../data/ocsai1/finetune-gt_main2_prepared_val.jsonl')
print(trainpath, valpath)

../data/ocsai1/finetune-gt_main2_prepared_train.jsonl ../data/ocsai1/finetune-gt_main2_prepared_val.jsonl


Convert format from 

`{'prompt': 'AUT Prompt:hat\nResponse:Use it as a kite in the air\nScore:\n',
  'completion': ' 39'}`

to

`{"messages": [{"role": "system", "content": "You score originality in the alternate uses task."}, {"role": "user", "content": "AUT Prompt:hat\nResponse:Use it as a kite in the air"}, {"role": "assistant", "content": "39"}]}`

In [6]:
def rewrite_legacy_to_chat(path):
    outpath = path.parent / (path.stem+'_chat.jsonl')
    with open(path) as f:
        train = [json.loads(line) for line in f]

    with open(outpath, mode='w') as f:
        for i, line in enumerate(train):
            msgs = {"messages": [
                    {"role": "system", "content": "You score originality in the alternate uses task."},
                    {"role": "user", "content": line['prompt'].split('\nScore:')[0] },
                    {"role": "assistant", "content": line['completion'].strip()}
                    ]
                }
            f.write(json.dumps(msgs))
            if i < len(train)-1:
                f.write('\n')
rewrite_legacy_to_chat(trainpath)
rewrite_legacy_to_chat(valpath)

### Compare sizes

To get a sense of cost for Replicate, compare the train size to the ABC Notation dataset. (https://replicate.com/docs/guides/fine-tune-a-language-model)

In [22]:
with open(trainpath) as f:
    train = [json.loads(line) for line in f]

with open('/Users/peter.organisciak/Downloads/massive_abcnotation_dataset.jsonl') as f:
    abc = [json.loads(line) for line in f]

print("proportion of examples", len(train)/len(abc))

train_chars = sum([len(ex['prompt']+ex['completion']) for ex in train])
abc_chars = sum([len(ex['prompt']+ex['completion']) for ex in abc])
print("proportion of chars", train_chars / abc_chars)

proportion of examples 0.080405
proportion of chars 0.017347935290594346
